# Chapter 4: Tactical portfolio management

In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# plt.style.use('seaborn')
plt.style.use('seaborn-v0_8')
import warnings
warnings.filterwarnings("ignore")
import yfinance as yf
from datetime import date

In [11]:
todays_date = date.today()
# Importation of data
# list_tickers = ["FB", "NFLX", "TSLA"]
list_tickers = ["META", "NFLX", "TSLA"]
database = yf.download(tickers=list_tickers,
                       start='2012-01-01',
                       # end=todays_date,
                       end='2021-12-31',
                       auto_adjust=False,
                       multi_level_index=False
                      )
database

[*********************100%***********************]  3 of 3 completed


Price        Adj Close                               Close              \
Ticker            META        NFLX        TSLA        META        NFLX   
Date                                                                     
2012-01-03         NaN   10.320000    1.872000         NaN   10.320000   
2012-01-04         NaN   11.492857    1.847333         NaN   11.492857   
2012-01-05         NaN   11.328571    1.808000         NaN   11.328571   
2012-01-06         NaN   12.327143    1.794000         NaN   12.327143   
2012-01-09         NaN   14.025714    1.816667         NaN   14.025714   
...                ...         ...         ...         ...         ...   
2021-12-23  333.410858  614.090027  355.666656  335.239990  614.090027   
2021-12-27  344.291199  613.119995  364.646667  346.179993  613.119995   
2021-12-28  344.331024  610.710022  362.823334  346.220001  610.710022   
2021-12-29  341.068848  610.539978  362.063324  342.940002  610.539978   
2021-12-30  342.481110  612.090027  356.779999  344.359985  612.090027   

Price                         High                                 Low  \
Ticker            TSLA        META        NFLX        TSLA        META   
Date                                                                     
2012-01-03    1.872000         NaN   10.388571    1.966667         NaN   
2012-01-04    1.847333         NaN   11.567143    1.911333         NaN   
2012-01-05    1.808000         NaN   11.694286    1.862000         NaN   
2012-01-06    1.794000         NaN   12.471429    1.852667         NaN   
2012-01-09    1.816667         NaN   14.235714    1.832667         NaN   
...                ...         ...         ...         ...         ...   
2021-12-23  355.666656  336.670013  616.880005  357.660004  328.359985   
2021-12-27  364.646667  347.869995  615.000000  372.333344  338.010010   
2021-12-28  362.823334  352.709991  618.409973  373.000000  345.200012   
2021-12-29  362.063324  349.690002  613.979980  368.000000  341.640015   
2021-12-30  356.779999  347.230011  620.609985  365.183319  343.220001   

Price                                     Open                          \
Ticker            NFLX        TSLA        META        NFLX        TSLA   
Date                                                                     
2012-01-03   10.018571    1.843333         NaN   10.040000    1.929333   
2012-01-04   10.258571    1.833333         NaN   10.335714    1.880667   
2012-01-05   11.040000    1.790000         NaN   11.345714    1.850667   
2012-01-06   11.242857    1.760667         NaN   11.302857    1.813333   
2012-01-09   12.448571    1.741333         NaN   12.801429    1.800000   
...                ...         ...         ...         ...         ...   
2021-12-23  607.570007  332.519989  330.100006  616.400024  335.600006   
2021-12-27  609.250000  356.906677  338.850006  615.000000  357.890015   
2021-12-28  609.690002  359.473328  346.630005  614.950012  369.829987   
2021-12-29  604.679993  354.713318  346.910004  610.710022  366.213318   
2021-12-30  611.239990  351.049988  344.000000  612.989990  353.776672   

Price           Volume                       
Ticker            META       NFLX      TSLA  
Date                                         
2012-01-03         NaN   29213800  13921500  
2012-01-04         NaN  100489900   9451500  
2012-01-05         NaN   86552900  15082500  
2012-01-06         NaN  125610800  14794500  
2012-01-09         NaN  214073300  13455000  
...                ...        ...       ...  
2021-12-23  13987700.0    1621100  92713200  
2021-12-27  17795000.0    2061500  71145900  
2021-12-28  16637600.0    1882800  60324000  
2021-12-29  10747000.0    1287200  56154000  
2021-12-30  10593300.0    1625100  47040900  

[2516 rows x 18 columns]

In [12]:
# Take only the adjusted stock price
database = database["Adj Close"]
database

Ticker,META,NFLX,TSLA
Date,,,
2012-01-03,NaN,10.320000,1.872000
2012-01-04,NaN,11.492857,1.847333
2012-01-05,NaN,11.328571,1.808000
2012-01-06,NaN,12.327143,1.794000
2012-01-09,NaN,14.025714,1.816667
...,...,...,...
2021-12-23,333.410858,614.090027,355.666656
2021-12-27,344.291199,613.119995,364.646667
2021-12-28,344.331024,610.710022,362.823334


In [13]:
# Drop missing values
data = database.dropna()
data

Ticker,META,NFLX,TSLA
Date,,,
2012-05-18,38.021412,9.994286,1.837333
2012-05-21,33.844326,10.248571,1.918000
2012-05-22,30.830860,9.672857,2.053333
2012-05-23,31.825403,10.272857,2.068000
2012-05-24,32.849781,10.038571,2.018667
...,...,...,...
2021-12-23,333.410858,614.090027,355.666656
2021-12-27,344.291199,613.119995,364.646667
2021-12-28,344.331024,610.710022,362.823334
